## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `parentRAG.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [94]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [95]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [96]:
# Carregar modelos Open AI - Embedding e chat

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=500,
)

In [97]:
# Carregar o PDF
pdf_link = "DOC-SF238339076816-20230503.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [98]:
len(pages)

31

In [99]:
# Separar em chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [100]:
# Salvar os Chunks no VectorDB

vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="naiveDB")

In [101]:
# Carregar o DB

naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [102]:
rerank = CohereRerank(model="rerank-v3.5", top_n=3)

compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=naive_retriever,
)


In [29]:
TEMPLATE = """
    Você é um especialista em legislação e tecnologia. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}
    Context:
    {context}
"""
rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [31]:
setup_retrival = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})
output_parser = StrOutputParser()

compressor_retrieval_chain = setup_retrival | rag_prompt | llm | output_parser

In [32]:
compressor_retrieval_chain.invoke("Quais os principais riscos do marco legal de ia")

'Como não há um contexto específico fornecido, posso destacar alguns dos principais riscos do marco legal de IA (Inteligência Artificial) que geralmente são discutidos:\n\n1. Violação da privacidade: O uso de IA pode envolver a coleta e processamento de grandes quantidades de dados pessoais, o que pode resultar em violações de privacidade se não houver regulamentações adequadas para proteger essas informações.\n\n2. Discriminação algorítmica: Algoritmos de IA podem reproduzir e amplificar preconceitos existentes, resultando em decisões discriminatórias em áreas como contratação, empréstimos, justiça criminal, entre outros.\n\n3. Responsabilidade e transparência: A complexidade dos sistemas de IA pode dificultar a atribuição de responsabilidade em caso de erros ou danos causados \u200b\u200bpor decisões automatizadas. Além disso, a falta de transparência nos algoritmos pode dificultar a compreensão de como as decisões são tomadas.\n\n4. Segurança cibernética: A integração de sistemas de